In [ ]:
import psycopg2
import matplotlib.pyplot as plt

try:
    # Подключение к базе данных
    conn = psycopg2.connect(
        user='bk_465842_2025',
        password='bk_465842',
        host='postgrepro.dc-edu.ru',
        port='5432',
        database='dbstud'
    )
    cursor = conn.cursor()

    def cost_by_client_id():
      ex_str = """
      SELECT cost
      FROM order_details
      JOIN taxi_order ON taxi_order.order_id = order_details.order_id
      WHERE taxi_order.client_id = 3
      """
      cursor.execute(ex_str)
      results = cursor.fetchall()
      for row in results:
        for i in row:
          print(f"цена заказа клиента с id = 3: {i}")

    def order_cost_by_month():
        ex_str = """
        SELECT COUNT(cost) as order_count
        FROM order_details
        JOIN taxi_order ON taxi_order.order_id = order_details.order_id
        WHERE EXTRACT(MONTH FROM TO_TIMESTAMP(order_datetime, 'DD:MM:YY HH24:MI:SS')) = 5
        """
        cursor.execute(ex_str)
        result = cursor.fetchone()
        print(f"Количество заказов в мае: {result[0]}")

    def car_colors():
      ex_str = """
        SELECT DISTINCT car_color FROM driver ORDER BY car_color
      """
      cursor.execute(ex_str)
      return [color[0] for color in cursor.fetchall()]

    def count_order_status():
      cursor.execute("""
      SELECT
          status,
          COUNT(*) as count
      FROM order_details
      GROUP BY status
      ORDER BY count DESC
      """)
      result = cursor.fetchall()
      print(f"Количество заказов ('тип', количество): {result}")

    def taxi_orders_by_hours():
      ex_str = """
      SELECT
          EXTRACT(HOUR FROM TO_TIMESTAMP(order_datetime, 'DD:MM:YY HH24:MI:SS'))::INTEGER as hour,
          COUNT(*)::INTEGER as order_count
      FROM taxi_order
      GROUP BY hour
      ORDER BY hour
      """
      cursor.execute(ex_str)
      result = cursor.fetchall()
      print(f"Заказы такси по часам (часб количество): {result}")

    def payment_type():
      ex_str = """
      SELECT client_name FROM client
      JOIN taxi_order ON client.client_id = taxi_order.client_id
      JOIN order_details ON order_details.order_id = taxi_order.order_id
      WHERE order_details.payment_type IN ('наличные')
      """
      cursor.execute(ex_str)
      result = cursor.fetchall()
      print(f"Список склиентов, плативших картой: {result}")

    def order_by_days():
      ex_str = """
        SELECT
            TO_CHAR(TO_TIMESTAMP(order_datetime, 'DD:MM:YY HH24:MI:SS'), 'DD-MM-YY') AS day_month_year,
            SUM(order_details.cost) AS total_cost
        FROM taxi_order
        JOIN order_details ON taxi_order.order_id = order_details.order_id
        GROUP BY day_month_year
        ORDER BY day_month_year;
        """

      cursor.execute(ex_str)
      data = cursor.fetchall()
      print(f"Сумма заказов за день: {data}")

    order_cost_by_month() #
    cost_by_client_id() #
    count_order_status() #
    colors = car_colors() #
    print("Список цветов:", colors)
    taxi_orders_by_hours() #
    payment_type() #
    order_by_days()


except Exception as e:
    print(f"Ошибка: {e}")
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()